In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Get the current working directory
current_dir = os.getcwd()
# Move up one level from the current directory
parent_dir = os.path.dirname(current_dir)
# Change directory into data directory
data_dir = os.path.join(parent_dir, 'data')

In [3]:
print(data_dir)

c:\Users\hamin\Desktop\Supply-Chain-Analytic\data


# Extract

In [4]:
df_fulfillment = pd.read_csv(os.path.join(data_dir, 'fulfillment.csv'), index_col=None)
df_inventory = pd.read_csv(os.path.join(data_dir, 'inventory.csv'), index_col=None)
df_orders = pd.read_csv(os.path.join(data_dir, 'orders_and_shipments.csv'), index_col=None)

In [5]:
df_fulfillment.head()

,Product Name,Warehouse Order Fulfillment (days)
0,Perfect Fitness Perfect Rip Deck,8.3
1,Nike Men's Dri-FIT Victory Golf Polo,6.6
2,O'Brien Men's Neoprene Life Vest,5.5
3,Nike Men's Free 5.0+ Running Shoe,9.4
4,Under Armour Girls' Toddler Spine Surge Runni,6.3


# 2.Transform

## 2.1 Rename Columns

In [6]:
def rename_columns(df):
    # Convert from camel case to snake case
    df.columns = (
        df.columns
        .str.replace('(?<=[a-z])(?=[A-Z])', '_', regex=True)
        .str.lower()
    )
    # Replace spaces with underscores
    df.columns = df.columns.str.replace(' ', '_')
    # Replace hyphens with underscores
    df.columns = df.columns.str.replace('-', '_')
    # Replace percentage signs with 'percent'
    df.columns = df.columns.str.replace('%', 'percent')
    # Remove parentheses
    df.columns = df.columns.str.replace('[()]', '', regex=True)
    # Replace multiple underscores with a single underscore
    df.columns = df.columns.str.replace('_+', '_', regex=True)
    # Remove leading and trailing underscores
    df.columns = df.columns.str.strip('_')
    
    return df

In [7]:
df_inventory = rename_columns(df_inventory)
df_fulfillment = rename_columns(df_fulfillment)
df_orders = rename_columns(df_orders)

In [8]:
df_fulfillment.columns

Index(['product_name', 'warehouse_order_fulfillment_days'], dtype='object')

## 2.2 Check Datatypes

In [9]:
df_fulfillment.dtypes

product_name                         object
warehouse_order_fulfillment_days    float64
dtype: object

In [10]:
df_inventory.dtypes

product_name                object
year_month                   int64
warehouse_inventory          int64
inventory_cost_per_unit    float64
dtype: object

In [11]:
df_orders.dtypes

order_id                    int64
order_item_id               int64
order_year_month            int64
order_year                  int64
order_month                 int64
order_day                   int64
order_time                 object
order_quantity              int64
product_department         object
product_category           object
product_name               object
customer_id                 int64
customer_market            object
customer_region            object
customer_country           object
warehouse_country          object
shipment_year               int64
shipment_month              int64
shipment_day                int64
shipment_mode              object
shipment_days_scheduled     int64
gross_sales                 int64
discount_percent           object
profit                      int64
dtype: object

# 3. Load

In [12]:
import pyodbc
from sqlalchemy import create_engine
import sqlalchemy

In [13]:
# Define the connection string using Windows Authentication
conn = pyodbc.connect(
    'DRIVER={SQL Server};'
    'SERVER=BOOK-3EOH2MPGSA\SQLEXPRESS;'
    'DATABASE=SUPPLY_CHAIN_ANALYTICS;'
    'Trusted_Connection=yes;'
)

<>:4: SyntaxWarning: invalid escape sequence '\S'
<>:4: SyntaxWarning: invalid escape sequence '\S'
C:\Users\hamin\AppData\Local\Temp\ipykernel_7372\3518523648.py:4: SyntaxWarning: invalid escape sequence '\S'
  'SERVER=BOOK-3EOH2MPGSA\SQLEXPRESS;'


In [14]:
def get_column_types(df):
    # Map Pandas data types to SQL Server data types
    type_map = {
        'object': 'NVARCHAR(MAX)',
        'int64': 'BIGINT',
        'float64': 'FLOAT',
        'datetime64[ns]': 'DATETIME2',
        'bool': 'BIT'
    }
    return [type_map.get(str(dt), 'NVARCHAR(MAX)') for dt in df.dtypes]

In [15]:
def load_table(df, table_name, schema_name):
    # Define the connection string using Windows Authentication
    conn_str = (
        'Driver={SQL Server};'
        'Server=BOOK-3EOH2MPGSA\\SQLEXPRESS;'
        'Database=SUPPLY_CHAIN_ANALYTICS;'
        'Trusted_Connection=yes;'
    )

    # Create a pyodbc connection
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Drop the table if it already exists
    cursor.execute(f"IF OBJECT_ID('{schema_name}.{table_name}', 'U') IS NOT NULL DROP TABLE {schema_name}.{table_name}")
    conn.commit()

    # Create the table schema
    columns = ', '.join([f'{col} {dtype}' for col, dtype in zip(df.columns, get_column_types(df))])
    create_table_sql = f"CREATE TABLE {schema_name}.{table_name} ({columns})"
    cursor.execute(create_table_sql)
    conn.commit()

    # Insert the data into the table
    insert_sql = f"INSERT INTO {schema_name}.{table_name} VALUES ({','.join(['?'] * len(df.columns))})"
    for _, row in df.iterrows():
        cursor.execute(insert_sql, *row.tolist())
    conn.commit()

    cursor.close()
    conn.close()
    print(f"DataFrame loaded successfully into {schema_name}.{table_name}")

In [16]:
df_fulfillment.dtypes

product_name                         object
warehouse_order_fulfillment_days    float64
dtype: object

In [17]:
load_table(df_fulfillment, 'fulfillment', 'original')

DataFrame loaded successfully into original.fulfillment


In [18]:
load_table(df_orders, 'orders', 'original')

DataFrame loaded successfully into original.orders


In [19]:
load_table(df_inventory, 'inventory', 'original')

DataFrame loaded successfully into original.inventory


# 4. Create and Load dim_date

In [25]:
# Step 1: Generate the date range
date_range = pd.date_range(start='2021-01-01', end='2023-12-31')

# Step 2: Create the DataFrame
df_date = pd.DataFrame({
    'date': date_range,
    'year': date_range.year,
    'quarter': date_range.quarter,
    'month': date_range.month,
    'month_name': date_range.strftime('%B'),
    'month_abbreviation': date_range.strftime('%b'),
    'day': date_range.day,
    'day_of_week': date_range.weekday + 2,  # Monday=1, Sunday=7
    'day_name': date_range.strftime('%A'),
    'day_abbreviation': date_range.strftime('%a'),
    'week': date_range.isocalendar().week,
    'is_weekend': date_range.weekday >= 5,  # 5=Saturday, 6=Sunday
    'fiscal_year': date_range.year,  # Adjust if fiscal year differs
    'fiscal_quarter': date_range.quarter,
    'fiscal_month': date_range.month,
    'is_last_day_of_month': date_range.isin(pd.date_range(start='2021-01-01', end='2023-12-31', freq='M')),
    'is_last_day_of_quarter': date_range.isin(pd.date_range(start='2021-01-01', end='2023-12-31', freq='Q')),
    'is_last_day_of_year': date_range.isin(pd.date_range(start='2021-01-01', end='2023-12-31', freq='A'))
})

C:\Users\hamin\AppData\Local\Temp\ipykernel_7372\2449815114.py:21: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  'is_last_day_of_month': date_range.isin(pd.date_range(start='2021-01-01', end='2023-12-31', freq='M')),
C:\Users\hamin\AppData\Local\Temp\ipykernel_7372\2449815114.py:22: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  'is_last_day_of_quarter': date_range.isin(pd.date_range(start='2021-01-01', end='2023-12-31', freq='Q')),
C:\Users\hamin\AppData\Local\Temp\ipykernel_7372\2449815114.py:23: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  'is_last_day_of_year': date_range.isin(pd.date_range(start='2021-01-01', end='2023-12-31', freq='A'))


In [26]:
df_date.head()

,date,year,quarter,month,month_name,month_abbreviation,day,day_of_week,day_name,day_abbreviation,week,is_weekend,fiscal_year,fiscal_quarter,fiscal_month,is_last_day_of_month,is_last_day_of_quarter,is_last_day_of_year
2021-01-01,2021-01-01,2021,1,1,January,Jan,1,6,Friday,Fri,53,False,2021,1,1,False,False,False
2021-01-02,2021-01-02,2021,1,1,January,Jan,2,7,Saturday,Sat,53,True,2021,1,1,False,False,False
2021-01-03,2021-01-03,2021,1,1,January,Jan,3,8,Sunday,Sun,53,True,2021,1,1,False,False,False
2021-01-04,2021-01-04,2021,1,1,January,Jan,4,2,Monday,Mon,1,False,2021,1,1,False,False,False
2021-01-05,2021-01-05,2021,1,1,January,Jan,5,3,Tuesday,Tue,1,False,2021,1,1,False,False,False


In [27]:
load_table(df_date, 'dim_date', 'dbo')

DataFrame loaded successfully into dbo.dim_date
